In [2]:
import numpy as np
import scipy # use numpy if scipy unavailable
import scipy.linalg # use numpy if scipy unavailable
import random
import math
import time
import numpy as np
import matplotlib
import matplotlib.animation as animation
#matplotlib.use('TkAgg') # do this before importing pylab
import matplotlib.pyplot as plt 
import json
from pprint import pprint
import copy
import dates

ImportError: No module named dates

In [4]:
#HELPER FUNCTION
def lon_lat_arc_distance(lat1, long1, lat2, long2):
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
 
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc * 6371 #km

def import_time_data(name = '2men.txt'):
    ids = {} #{id: timestamp}
    time = []
    ratio = []
    with open(name, "r") as f:
        for line in f:
            words = line.split(',')
            i = 0
            while i < (len(words)-1):
                ratio.append(i * 1.0 / len(words))
                time.append(words[i+1])
                ids[words[i]] = words[i+1]
                i += 2
    return ids, time, ratio

def gauss_2d(x0, y0, n):
    mean = [x0, y0]
    cov = [[2, 0], [0, 2]]

    x,y = np.random.multivariate_normal(mean, cov, n).T
    return zip(x,y)

In [3]:
#CLASSES
class Network:
    def __init__(self):
        self.nodes = {}
        self.infected_nodes_ids = []
        self.non_infected_nodes_ids = []
        self.t = 0
        self.K = 0   #how much distance affects dispersion
        self.A = 1 #captivating measure of information
        self.I = 10  #importance measure
    
    #input:
    #node_id - name of the node
    #node_edges_ids - array of vertices id's that current node links to
    def add_node(self, node):
        node_id = node.get_id()
        input_ids = node.get_input_ids()
        output_ids = node.get_output_ids()
        if node.get_infected():
            self.infected_nodes_ids.append(node_id)
        else:
            self.non_infected_nodes_ids.append(node_id)

    def init_nodes(self, nodes, K, A, I):
        self.K = K
        self.A = A
        self.I = I
        self.infected_nodes_ids = []
        self.non_infected_nodes_ids = []
        self.t = 0
        self.nodes = copy.deepcopy(nodes)
        for node_id, node in nodes.iteritems():
            self.add_node(node)
        print "nodes initialized"

    #step function for network, dt - time step 
    def step(self, dt):
        self.t += dt
        print self.t

        K = 0   #how much distance affects dispersion
        A = 1 #captivating measure of information
        I = 10  #importance measure

        a_map = {} #alpha hashmap of {(i,j): edge_value}
        l_map = {} #lambda hashmpa of {i: lambda_rate(t) value}

        #for every i -> n (infected -> non_infected) edge, compute a(i,j) parameter
        for j in self.non_infected_nodes_ids:
            j_input = self.nodes[j].get_input_ids()
            l_j = 0 #lambda(i)
            for i in j_input:
                if self.nodes[i].get_infected(): #only infected neighbours affect rate
                    d = lon_lat_arc_distance(self.nodes[j].get_x(), self.nodes[j].get_y(), self.nodes[i].get_x(), self.nodes[i].get_y())  #distance between two point
                    t_i_decay = self.t - self.nodes[i].get_infection_time()
                    l_j += A*math.exp(- t_i_decay / I) / (K*d**2 + 1)
            
            if self.was_infected(l_j, dt): #infect j if exponential dist-n returned True for given dt and lambda
                self.nodes[j].infect(self.t)
                self.infected_nodes_ids.append(j)
                self.non_infected_nodes_ids.remove(j)
            
    def was_infected(self, lam, dt):
        import random
        from scipy import exp
        rnd = random.random()
        if rnd <= 1-exp(-dt*lam):
            return True
        else:
            return False    
        
    def get_infection_ratio(self):
        return len(self.infected_nodes_ids)*100.0/len(self.nodes.keys()) 

    def print_network(self):
        edges = ''
        #for key, value in self.nodes.iteritems():
        #    edges += '\t\t\t{}, input_ids: {}, output_ids: {}\n'.format(key, value.get_input_ids(), value.get_output_ids())
        print " infected_nodes_ids: \t\t\t{} \n non_infected_nodes_ids: \t\t\t{} \n node_edges: \n{} \n infection_ratio: \t\t\t{}".format(self.infected_nodes_ids, self.non_infected_nodes_ids, edges, self.get_infection_ratio())
        print "#infected - {} \n#non_infected - {}".format(len(self.infected_nodes_ids), len(self.non_infected_nodes_ids))

    def get_nodes(self):
        return self.nodes
        

class Node:
    '''
    Input:
    size - population of the node
    status - 0:not infected, 1:infecting others'''
    def __init__(self, id, size=1, infection_status=0, input_ids=[], output_ids=[], x=0, y=0, time_from_infection=0):
        self.size = size
        self.id = id
        self.infected = infection_status
        self.output_ids = output_ids
        self.input_ids = input_ids
        self.x = x
        self.y = y
        self.time_from_infection = 0

    def get_infection_time(self):
        return self.time_from_infection

    def get_x(self):
        return self.x

    def get_y(self):
        return self.y

    def get_output_ids(self):
        return self.output_ids
    
    def get_input_ids(self):
        return self.input_ids

    def get_infected(self):
        return self.infected

    def set_output_ids(self, output_ids):
        self.output_ids = output_ids

    def set_input_ids(self, input_ids):
        self.input_ids = input_ids

    def infect(self, current_t):
        self.infected = 1
        self.time_from_infection = current_t
    
    def get_id(self):
        return self.id

In [26]:
#BUILDING GRAPHS
#connect node_ids randomly
def add_random_edges(nodes, n_edges):
    node_ids = nodes.keys()
    node_in_out_ids = {} # {node_id: input, output} - to ensure input_nodes, output_nodes consistency
    for node_id in node_ids:
        node_in_out_ids.setdefault(node_id, [[],[]])
    for node_id in node_ids:
        new_node_output = [int(random.random() * len(node_ids)) for i in range(n_edges)]
        #uniqueness
        for new_id in new_node_output:           
            if (new_id not in node_in_out_ids[node_id][1]) and (new_id != node_id): #if new output_id isn't already in output_ids, add it
                #adding i-j edge on both receiving and giving side
                node_in_out_ids[new_id][0].append(node_id) #receiving side  -> j
                node_in_out_ids[node_id][1].append(new_id) #giving side     i ->

    for node_id in node_ids:
        nodes[node_id].set_input_ids(node_in_out_ids[node_id][0])
        nodes[node_id].set_output_ids(node_in_out_ids[node_id][1])
    
    return nodes

#centers = {'new york': [x,y,frac, status],...}
def make_nodes(n, centers, infection_ratio = 0.01, mode = 'infect_all'):
    nodes = {}
    inf_coordinates = []
    non_inf_coordinates = []
    offsets = [0]
    #get coordinates
    print "getting coordinates"
    for city, center in centers.iteritems():
        #TODO: ratio of n's in different locations
        new_locations = gauss_2d(center[0], center[1], int(n*center[2]))
        if center[3] == 1:
            inf_coordinates += new_locations
        else:
            if mode == 'infect_all':
                inf_coordinates += new_locations
            else:
                non_inf_coordinates += new_locations
        offsets.append(new_locations)

    print "creating nodes"
    for i in range(len(inf_coordinates)):
        xy = inf_coordinates[i]
        if random.random() < infection_ratio:
            infection_status = 1
        else:
            infection_status = 0
        nodes[i] = Node(i, 1, infection_status, [], [], xy[0], xy[1])

    for i in range(len(non_inf_coordinates)):
        xy = non_inf_coordinates[i]
        nodes[i] = Node(i, 1, 0, [], [], xy[0], xy[1])
    #add edges
    print "random edgess"
    new_nodes = add_random_edges(nodes, 2)
    return new_nodes

def make_centers(infection_cities = ['New York']):
    with open('cities.json') as data_file:    
        data = json.load(data_file)
    pprint(data[0]['city'])
    cities = {} # {'New York': [x, y, fraction of total people]}
    total_count = 0

    for entry in data:
        total_count += int(entry['population'])

    for entry in data:
        x = float(entry['longitude'])
        y = float(entry['latitude'])
        #x =  - int((MAP_WIDTH/360.0) * (180 + float(entry['longitude'])))
        #y =   int((MAP_HEIGHT/180.0) * (90  - float(entry['latitude'])))
        frac = int(entry['population']) * 1.0 / total_count
        #if frac > 0.00000:
        city_name = str(entry['city'])
        if city_name in infection_cities:
            inf_status = 1
        else:
            inf_status = 0
        cities[city_name] = [x, y, frac, inf_status]
    return cities



In [27]:
def monte_carlo(centers, t_max, dt,  K, A, I, n = 10):
    print 'monte carlo'
    plt.axis([0,t_max,0,100])
    plt.title("Monte Carlo simulation")
    plt.xlabel("time")
    plt.ylabel("percent of people informed")
    nodes_init = make_nodes(1000, centers)
    for i in range(n):
        print 'monte carlo', i
        network = Network()
        network.init_nodes(nodes_init,  K, A, I)
        infection_growth, changed_nodes = simulate(network, t_max, dt, 'dont plot')
        plt.plot(np.arange(0, t_max, dt), infection_growth, linewidth=0.1, color='r')
    plt.show()

def simulate(network, time_length, dt, mode):
    x_all = []
    y_all = []
    color_all = []
    infection_ratio_all = []

    for i in range(int(time_length/dt)):
        changed_nodes = network.get_nodes()
        network.step(dt)

        x = []
        y = []
        color = []
        infection_ratio = network.get_infection_ratio()
        for node_id, node in changed_nodes.iteritems():
            x.append(node.get_x())
            y.append(node.get_y())
            if node.get_infected() == 1:
                color.append(0)
            else:
                color.append(1)
        x_all.append(x)
        y_all.append(y)
        color_all.append(color)
        infection_ratio_all.append(infection_ratio)
    
    if (mode == 'plot'):
        fig = plt.figure()
        scat = plt.scatter(x_all[0], y_all[0], c=color_all[0], s=5, alpha=0.9)
        colors = np.array(color_all)
        anim = animation.FuncAnimation(fig, update_plot, frames=1000,
                                      fargs=(colors, scat))
        plt.show()

    changed_nodes = network.get_nodes()
    return infection_ratio_all, changed_nodes

def update_plot(i, data, scat):
    scat.set_array(data[i])
    return scat,


In [30]:
def main():
    #dummy nodes
    dt = 1
    t_max = 30
    K = 10
    A = 1
    I = 0.1

    print "importing locations"
    centers = make_centers()
    nodes = make_nodes(1000, centers)
    network = Network()
    network.init_nodes(nodes,  K, A, I)

    print "simulating spread"
    monte_carlo(centers, t_max, dt,  K, A, I, 10)

main()

importing locations
u'New York'
getting coordinates
creating nodes
random edgess
nodes initialized
simulating spread
monte carlo
getting coordinates
creating nodes
random edgess
monte carlo 0
nodes initialized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
monte carlo 1
nodes initialized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
monte carlo 2
nodes initialized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
monte carlo 3
nodes initialized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
monte carlo 4
nodes initialized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
monte carlo 5
nodes initialized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
monte carlo 6
nodes initialized
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
monte carlo 7
nodes initialized

In [ ]:
from matplotlib import dates
import datetime

data, times, ratios = import_time_data()
new_times = []
for time in times:
    new_time = dates.num2date(dates.epoch2num(float(time) + 1298962800))
    new_times.append(new_time)


hfmt = dates.DateFormatter('%m/%d %H:%M')
fig, ax = plt.subplots()
plt.plot(new_times, ratios, c='r')
plt.ylabel(r'Informed Websites to Total Websites Tracked', fontsize=14)

fig.subplots_adjust(bottom=0.2)
ax.xaxis.set_major_formatter(hfmt)
fig.autofmt_xdate()


plt.show()